In [ ]:
#Import Bibliotecas
import pypsa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# ===========
# Zeitprofile
# ===========

# Lade das ninja_pv Solar-Profil für Deutschland (51.16°N, 13°E)
renewable_profile_solar = (pd.read_csv("pv_profile.csv", usecols=["electricity"]) ["electricity"].astype(float))  # Stündliches, normiertes PV-Erzeugungsprofil

# Building load als csv datei einlesen (kW → MW)
electric_load_profile = pd.read_csv("building_electric_load.csv", usecols=["Gebaeude_Strom_Gesamt_kW"])["Gebaeude_Strom_Gesamt_kW"].astype(float) / 1000  # In MW

# Building thermal load als csv datei einlesen (kW → MW)
thermal_load_bww = pd.read_csv("bww_load.csv", header=None)[1].astype(float) / 1000  # In MW
thermal_load_heating = pd.read_csv("building_heat_load.csv", header=None)[1].astype(float) / 1000  # In MW

# Temperatur profile
temperature_profile = pd.read_csv("temprature_profile.csv", header=None)[2].astype(float)

# Fleet load profile (kW → MW)
fleet_load_profile = pd.read_csv("fleet_load.csv", header=None)[1].astype(float) / 1000  # In MW

In [ ]:
# ===============================
#netzwerk erstellen with_batterie
# ===============================

with_battery = pypsa.Network()
with_battery.set_snapshots(range(8760))



In [ ]:
#hinzufügen bus
with_battery.add("Bus", name="electricity bus", carrier="electricity")
with_battery.add("Bus", name= "h_bus", carrier="heat")
with_battery.add("Bus", name= "h_bus2", carrier="heat")


In [ ]:
#hinzufügen generatoren
inhouse_pv = 0.35 #leistung der PV Anlage in MW
with_battery.add("Generator", name="PV", bus="electricity bus", p_nom=inhouse_pv, p_max_pu=renewable_profile_solar, marginal_cost=1)  # PV Betriebskosten: 1 €/MWh
with_battery.add("Generator", name="Grid", bus="electricity bus", p_nom_extendable=True, marginal_cost=200)  # Netzbezug: 150 €/MWh (15 Cent/kWh), fixe Leistung 10 MW


In [ ]:
#hinzufügen last
with_battery.add("Load", name="building_load", bus="electricity bus", p_set=electric_load_profile)
with_battery.add("Load", name="fleet_load", bus="electricity bus", p_set=fleet_load_profile)
with_battery.add("Load", name="DHW_Load",bus="h_bus", p_set=thermal_load_bww)
with_battery.add("Load", name="heating_Load",bus="h_bus2", p_set=thermal_load_heating)


In [ ]:
#wärmepumpen

# Umgebungstemperatur in °C
temp        = [-20, -15, -10,  -7,   2,   7,  10,  20,  30,  35] 
# Elektrische Leistungsaufnahme in kW 
el_power    = [10.2, 10.4, 10.6, 10.8, 11.2, 11.5, 11.7, 12.0, 12.2, 12.5] 
# Leistungszahl (COP) der Wärmepumpe bei 35°C Vorlauftemperatur
cop         = [1.65, 1.90, 2.20, 2.50, 3.40, 4.60, 5.10, 6.20, 7.10, 7.50]
# Nominale thermische Leistung (P_th) bei A7/W35 (Referenzpunkt)
hp_p_nom    = 39.5 # kW (thermisch)

# Interpolation für jede Stunde des Jahres
cop_profile = np.interp(temperature_profile, temp, cop)
el_p_pu_profile = np.interp(temperature_profile, temp, el_power) / hp_p_nom

# Wärmepumpe 1: Raumheizung (h_bus2)
with_battery.add("Link", name="h_pump", 
                 bus0="electricity bus", 
                 bus1="h_bus2", 
                 efficiency=cop_profile,  # Temperaturabhängig!
                 p_nom_extendable=True)

# Wärmepumpe 2: Warmwasser (h_bus)
with_battery.add("Link", name="h_pump2", 
                 bus0="electricity bus", 
                 bus1="h_bus", 
                 efficiency=cop_profile,  # Temperaturabhängig!
                 p_nom_extendable=True)



In [ ]:
#hinzufügen Wärmespeicher
with_battery.add("Store", name="DHW_Storage", bus="h_bus", carrier="heat", e_nom_extendable=True, e_cyclic=True, capital_cost=300000)  # 5 MWh Speicher, 0.5% Verlust/h
with_battery.add("Store", name="Thermal_Storage", bus="h_bus2", carrier="heat", e_nom_extendable=True, e_cyclic=True, capital_cost=300000)


In [ ]:
#hinzufügen Batteriespeicher
with_battery.add("Store", name="Battery", bus="electricity bus", carrier="electricity", e_nom_extendable=True, e_cyclic=True, capital_cost=400000)

In [ ]:
without_battery = with_battery.copy()
without_battery.remove("Store", "Battery")


In [ ]:
with_battery.optimize(solver_name='highs')
without_battery.optimize(solver_name='highs')

Index(['electricity bus', 'h_bus', 'h_bus2'], dtype='object', name='name')
Index(['h_pump', 'h_pump2'], dtype='object', name='name')
Index(['DHW_Storage', 'Thermal_Storage', 'Battery'], dtype='object', name='name')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 7/7 [00:00<00:00, 245.80it/s]
INFO:linopy.io: Writing time: 0.39s


Running HiGHS 1.12.0 (git hash: n/a): Copyright (c) 2025 HiGHS under MIT licence terms
LP linopy-problem-9ansncuj has 175206 rows; 87606 cols; 332886 nonzeros
Coefficient ranges:
  Matrix  [1e+00, 8e+00]
  Cost    [1e+00, 4e+05]
  Bound   [0e+00, 0e+00]
  RHS     [3e-09, 7e-01]
Presolving model
61320 rows, 65433 cols, 170550 nonzeros  0s
50275 rows, 54388 cols, 151881 nonzeros  0s
Dependent equations search running on 23419 equations with time limit of 1000.00s
Dependent equations search removed 0 rows and 0 nonzeros in 0.00s (limit = 1000.00s)
49699 rows, 53812 cols, 154394 nonzeros  0s
Presolve reductions: rows 49699(-125507); columns 53812(-33794); nonzeros 154394(-178492) 
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -5.6813243693e-03 Pr: 22849(6775.15) 0s


INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 87606 primals, 175206 duals
Objective: 6.32e+05
Solver model: available
Solver message: Optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Generator-ext-p-lower, Generator-ext-p-upper, Link-ext-p-lower, Link-ext-p-upper, Store-ext-e-lower, Store-ext-e-upper, Store-energy_balance were not assigned to the network.


      44376     6.3196796856e+05 Pr: 0(0) 1s

Performed postsolve
Solving the original LP from the solution after postsolve

Model name          : linopy-problem-9ansncuj
Model status        : Optimal
Simplex   iterations: 44376
Objective value     :  6.3196796850e+05
P-D objective error :  1.7500011670e-15
HiGHS run time      :          0.92


Index(['electricity bus', 'h_bus', 'h_bus2'], dtype='object', name='name')
Index(['h_pump', 'h_pump2'], dtype='object', name='name')
Index(['DHW_Storage', 'Thermal_Storage'], dtype='object', name='name')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 7/7 [00:00<00:00, 399.22it/s]
INFO:linopy.io: Writing time: 0.24s


Running HiGHS 1.12.0 (git hash: n/a): Copyright (c) 2025 HiGHS under MIT licence terms
LP linopy-problem-zzvbwrl0 has 148925 rows; 70085 cols; 271565 nonzeros
Coefficient ranges:
  Matrix  [1e+00, 8e+00]
  Cost    [1e+00, 3e+05]
  Bound   [0e+00, 0e+00]
  RHS     [3e-09, 7e-01]
Presolving model
43800 rows, 47912 cols, 117990 nonzeros  0s
35117 rows, 37098 cols, 93724 nonzeros  0s
Dependent equations search running on 16512 equations with time limit of 1000.00s
Dependent equations search removed 0 rows and 0 nonzeros in 0.00s (limit = 1000.00s)
33226 rows, 35178 cols, 92037 nonzeros  0s
Presolve reductions: rows 33226(-115699); columns 35178(-34907); nonzeros 92037(-179528) 
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     6.0739927885e+05 Pr: 16068(765.004) 0s


INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 70085 primals, 148925 duals
Objective: 6.32e+05
Solver model: available
Solver message: Optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Generator-ext-p-lower, Generator-ext-p-upper, Link-ext-p-lower, Link-ext-p-upper, Store-ext-e-lower, Store-ext-e-upper, Store-energy_balance were not assigned to the network.


      28732     6.3196796856e+05 Pr: 0(0) 0s
      28732     6.3196796856e+05 Pr: 0(0) 0s

Performed postsolve
Solving the original LP from the solution after postsolve

Model name          : linopy-problem-zzvbwrl0
Model status        : Optimal
Simplex   iterations: 28732
Objective value     :  6.3196796850e+05
P-D objective error :  1.7500011670e-15
HiGHS run time      :          0.48


('ok', 'optimal')

In [ ]:
print("batterie größe: ", with_battery.stores.at["Battery", "e_nom_opt"])
print("thermalstorage größe: ", with_battery.stores.at["Thermal_Storage", "e_nom_opt"])
print("DHW storage größe: ", with_battery.stores.at["DHW_Storage", "e_nom_opt"])

batterie größe:  -0.0
thermalstorage größe:  -0.0
DHW storage größe:  -0.0


In [ ]:
print(f"Grid Leistung: {with_battery.generators.loc['Grid', 'p_nom_opt']:.2f} MW")
print(f"PV Leistung: {with_battery.generators.loc['PV', 'p_nom_opt']:.2f} MW")
pv_energy = with_battery.generators_t.p["PV"].sum()  
print(f"PV Energieerzeugung über das Jahr: {pv_energy:.2f} MWh")    

print(f"Grid Leistung: {without_battery.generators.loc['Grid', 'p_nom_opt']:.2f} MW")
print(f"PV Leistung: {without_battery.generators.loc['PV', 'p_nom_opt']:.2f} MW")

Grid Leistung: 0.76 MW
PV Leistung: 0.35 MW
PV Energieerzeugung über das Jahr: 230.55 MWh
Grid Leistung: 0.76 MW
PV Leistung: 0.35 MW


In [ ]:
grid_energy_ohne = without_battery.generators_t.p["Grid"].sum()
grid_energy_mit = with_battery.generators_t.p["Grid"].sum()

print(f"Netzbezug ohne Batterie: {grid_energy_ohne:.1f} MWh")
print(f"Netzbezug mit Batterie: {grid_energy_mit:.1f} MWh")
print(f"Einsparung: {grid_energy_ohne - grid_energy_mit:.1f} MWh")

Netzbezug ohne Batterie: 3158.7 MWh
Netzbezug mit Batterie: 3158.7 MWh
Einsparung: 0.0 MWh


In [ ]:
# Elektrische Lasten
building_load = with_battery.loads_t.p["building_load"].sum()
fleet_load = with_battery.loads_t.p["fleet_load"].sum()

# Wärmepumpen-Stromverbrauch
h_pump_electricity = with_battery.links_t.p0["h_pump"].sum()  # Strom rein
h_pump2_electricity = with_battery.links_t.p0["h_pump2"].sum()  # Strom rein

# GESAMTER Stromverbrauch (direkt + Wärmepumpen)
total_electricity_demand = building_load + fleet_load + h_pump_electricity + h_pump2_electricity

# Autarkiegrad
pv_energy = with_battery.generators_t.p["PV"].sum()
autarky = (pv_energy / total_electricity_demand) * 100

print(f"PV-Erzeugung: {pv_energy:.1f} MWh")
print(f"Stromverbrauch (direkt): {building_load + fleet_load:.1f} MWh")
print(f"Stromverbrauch (Wärmepumpen): {h_pump_electricity + h_pump2_electricity:.1f} MWh")
print(f"Gesamter Stromverbrauch: {total_electricity_demand:.1f} MWh")
print(f"Autarkiegrad: {autarky:.1f}%")


PV-Erzeugung: 230.6 MWh
Stromverbrauch (direkt): 3252.1 MWh
Stromverbrauch (Wärmepumpen): 137.1 MWh
Gesamter Stromverbrauch: 3389.2 MWh
Autarkiegrad: 6.8%


In [ ]:
#Plots der Ergebnisse

#PV Erzeugung und Stromverbrauch über das Jahr
plt.figure(figsize=(12,6))
plt.plot(with_battery.generators_t.p["PV"].values, label='PV Erzeugung', linewidth=0.8)
plt.plot((with_battery.loads_t.p["building_load"] + with_battery.loads_t.p["fleet_load"]).values, label='Stromverbrauch (direkt)', linewidth=0.8)
plt.plot(with_battery.links_t.p0["h_pump"].values, label='Wärmepumpe 1 (Heizung)', linewidth=0.8)
plt.plot(with_battery.links_t.p0["h_pump2"].values, label='Wärmepumpe 2 (Warmwasser)', linewidth=0.8)
plt.xlabel('Zeit')
plt.ylabel('Leistung (kW)')
plt.title('PV Erzeugung und Stromverbrauch über das Jahr')
plt.legend()
plt.grid(True)
plt.show()